This demo introduces **Ray tune's** key concepts using a trivial examples. This example is derived from [Ray Tune basic example](https://docs.ray.io/en/latest/tune/examples/tune_basic_example.html). Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune.

 1. Setup your config space and define your trainable and objective function
 2. Use tune to execute your training, supplying the appropriate arguments including: search space, [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#blendsearch) or [trial schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers)
 3. Examine analyse the results
 
 <img src="https://docs.ray.io/en/latest/_images/tune-workflow.png" height="50%" width="60%">


See also the [Hyperparameter Tuning References](References-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


In [1]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

from ray import tune

## Step 1: Define our 'Trainable' training function to use with Ray Tune `ray.tune(...)`

In [2]:
NUM_OF_ACTORS = 4           # degree of parallel trials; each actor will have a separate trial
NUM_OF_CPUS_PER_ACTOR = 1   # number of CPUs per actor

In [3]:
ray_params = RayParams(num_actors=NUM_OF_ACTORS, cpus_per_actor=NUM_OF_CPUS_PER_ACTOR)

In [4]:
def train_func_model(config:dict):
    # create the dataset
    train_X, train_y = load_breast_cancer(return_X_y=True)
    # Convert to RayDMatrix data structure
    train_set = RayDMatrix(train_X, train_y)

    # Empty dictionary for the evaluation results reported back
    # to tune
    evals_result = {}

    # Train the model with XGBoost train
    bst = train(
        params=config,                       # our hyperparameter search space
        dtrain=train_set,                    # our RayDMatrix data structure
        evals_result=evals_result,           # place holder for results
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)                # distributed parameters configs for Ray Tune

    bst.save_model("model.xgb")

## Step 2: Define our hyperparameter search space

In [5]:
 # Specify the hyperparameter search space
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

## Step 3: Run Ray tune main trainer and examine the results

Ray Tune will launch distributed HPO, using four remote actors, each with its own instance of the trainable func

<img src="images/ray_tune_dist_hpo.png" height="50%" width="60%"> 

In [6]:
# Run tune
analysis = tune.run(
    train_func_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources()
)

2022-01-04 16:58:20,958	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265
2022-01-04 16:58:22,278	WARNING function_runner.py:561 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,eta,max_depth,subsample
train_func_model_946ba_00000,RUNNING,127.0.0.1:96814,0.000321294,1,0.586609
train_func_model_946ba_00001,PENDING,,0.00597151,2,0.939981
train_func_model_946ba_00002,PENDING,,0.000160446,5,0.580016
train_func_model_946ba_00003,PENDING,,0.00078595,3,0.577505


(ImplicitFunc pid=96814) 2022-01-04 16:58:24,456	INFO main.py:976 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=96814) 2022-01-04 16:58:25,897	INFO main.py:1021 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=96805) [16:58:25] task [xgboost.ray]:140659783761488 got new rank 1
(_RemoteRayXGBoostActor pid=96809) [16:58:25] task [xgboost.ray]:140472284872272 got new rank 3
(_RemoteRayXGBoostActor pid=96802) [16:58:25] task [xgboost.ray]:140407050698320 got new rank 0
(_RemoteRayXGBoostActor pid=96810) [16:58:25] task [xgboost.ray]:140423158464080 got new rank 2
(ImplicitFunc pid=96813) 2022-01-04 16:58:26,029	INFO main.py:976 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.


Result for train_func_model_946ba_00000:
  date: 2022-01-04_16-58-27
  done: false
  experiment_id: a85df0dd23de453facee9589f058cf26
  hostname: Juless-MacBook-Pro-16-inch-2019
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 96814
  time_since_restore: 2.904254913330078
  time_this_iter_s: 2.904254913330078
  time_total_s: 2.904254913330078
  timestamp: 1641344307
  timesteps_since_restore: 0
  train-error: 0.080844
  train-logloss: 0.692922
  training_iteration: 1
  trial_id: 946ba_00000
  
Result for train_func_model_946ba_00000:
  date: 2022-01-04_16-58-27
  done: true
  experiment_id: a85df0dd23de453facee9589f058cf26
  experiment_tag: 0_eta=0.00032129,max_depth=1,subsample=0.58661
  hostname: Juless-MacBook-Pro-16-inch-2019
  iterations_since_restore: 10
  node_ip: 127.0.0.1
  pid: 96814
  time_since_restore: 2.9658071994781494
  time_this_iter_s: 0.005114078521728516
  time_total_s: 2.9658071994781494
  timestamp: 1641344307
  timesteps_since_restore: 0
  train-error: 0.

(ImplicitFunc pid=96814) 2022-01-04 16:58:27,378	INFO main.py:1500 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 2.96 seconds (1.48 pure XGBoost training time).
(ImplicitFunc pid=96813) 2022-01-04 16:58:28,093	INFO main.py:1021 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=96806) [16:58:28] task [xgboost.ray]:140620456885888 got new rank 1
(_RemoteRayXGBoostActor pid=96808) [16:58:28] task [xgboost.ray]:140577641102976 got new rank 0
(_RemoteRayXGBoostActor pid=96855) [16:58:28] task [xgboost.ray]:140182641278544 got new rank 3
(_RemoteRayXGBoostActor pid=96854) [16:58:28] task [xgboost.ray]:140531336576592 got new rank 2


...

(ImplicitFunc pid=96858) 2022-01-04 16:58:33,364	INFO main.py:1500 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.94 seconds (1.61 pure XGBoost training time).
(ImplicitFunc pid=96868) 2022-01-04 16:58:34,296	INFO main.py:1021 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=96879) [16:58:34] task [xgboost.ray]:140631329472080 got new rank 0
(_RemoteRayXGBoostActor pid=96882) [16:58:34] task [xgboost.ray]:140532279606864 got new rank 3
(_RemoteRayXGBoostActor pid=96880) [16:58:34] task [xgboost.ray]:140490401054288 got new rank 1
(_RemoteRayXGBoostActor pid=96881) [16:58:34] task [xgboost.ray]:140355780349520 got new rank 2


Result for train_func_model_946ba_00003:
  date: 2022-01-04_16-58-35
  done: false
  experiment_id: c55cee2883014e62ae17edd794dd7d63
  hostname: Juless-MacBook-Pro-16-inch-2019
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 96868
  time_since_restore: 3.5583560466766357
  time_this_iter_s: 3.5583560466766357
  time_total_s: 3.5583560466766357
  timestamp: 1641344315
  timesteps_since_restore: 0
  train-error: 0.050967
  train-logloss: 0.692478
  training_iteration: 1
  trial_id: 946ba_00003
  
Result for train_func_model_946ba_00003:
  date: 2022-01-04_16-58-35
  done: true
  experiment_id: c55cee2883014e62ae17edd794dd7d63
  experiment_tag: 3_eta=0.00078595,max_depth=3,subsample=0.57751
  hostname: Juless-MacBook-Pro-16-inch-2019
  iterations_since_restore: 10
  node_ip: 127.0.0.1
  pid: 96868
  time_since_restore: 3.6066372394561768
  time_this_iter_s: 0.004676103591918945
  time_total_s: 3.6066372394561768
  timestamp: 1641344315
  timesteps_since_restore: 0
  train-error:

(ImplicitFunc pid=96868) 2022-01-04 16:58:35,555	INFO main.py:1500 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.60 seconds (1.25 pure XGBoost training time).


Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_func_model_946ba_00000,TERMINATED,127.0.0.1:96814,0.000321294,1,0.586609,10,2.96581,0.690905,0.070299
train_func_model_946ba_00001,TERMINATED,127.0.0.1:96813,0.00597151,2,0.939981,10,3.51433,0.646846,0.042179
train_func_model_946ba_00002,TERMINATED,127.0.0.1:96858,0.000160446,5,0.580016,10,3.94002,0.691782,0.024605
train_func_model_946ba_00003,TERMINATED,127.0.0.1:96868,0.00078595,3,0.577505,10,3.60664,0.686583,0.031634


2022-01-04 16:58:35,691	INFO tune.py:626 -- Total run time: 13.41 seconds (12.68 seconds for the tuning loop).


In [7]:
print("Best hyperparameters", analysis.best_config)

Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.00016044584783139323, 'subsample': 0.5800161276771466, 'max_depth': 5}


In [8]:
analysis.results_df.head(5)

/usr/local/anaconda3/lib/python3.8/site-packages/ray/tune/analysis/experiment_analysis.py:262: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


,train-logloss,train-error,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,...,iterations_since_restore,experiment_tag,config.tree_method,config.objective,config.eval_metric,config.eta,config.subsample,config.max_depth,config.nthread,config.n_jobs
trial_id,,,,,,,,,,,,,,,,,,,,,
946ba_00000,0.690905,0.070299,0.005114,True,None,None,10,a85df0dd23de453facee9589f058cf26,2022-01-04_16-58-27,1641344307,...,10,"0_eta=0.00032129,max_depth=1,subsample=0.58661",approx,binary:logistic,"[logloss, error]",0.000321,0.586609,1,1,1
946ba_00001,0.646846,0.042179,0.007070,True,None,None,10,f927df1b0ee64c198586fc72f7deba68,2022-01-04_16-58-29,1641344309,...,10,"1_eta=0.0059715,max_depth=2,subsample=0.93998",approx,binary:logistic,"[logloss, error]",0.005972,0.939981,2,1,1
946ba_00002,0.691782,0.024605,0.005455,True,None,None,10,cab894eb3b6e449b85a7ba6765df5ac4,2022-01-04_16-58-33,1641344313,...,10,"2_eta=0.00016045,max_depth=5,subsample=0.58002",approx,binary:logistic,"[logloss, error]",0.000160,0.580016,5,1,1
946ba_00003,0.686583,0.031634,0.004676,True,None,None,10,c55cee2883014e62ae17edd794dd7d63,2022-01-04_16-58-35,1641344315,...,10,"3_eta=0.00078595,max_depth=3,subsample=0.57751",approx,binary:logistic,"[logloss, error]",0.000786,0.577505,3,1,1


---

## References

 * [Ray Train: Tune: Scalable Hyperparameter Tuning](https://docs.ray.io/en/master/tune/index.html)
 * [Introducing Distributed XGBoost Training with Ray](https://www.anyscale.com/blog/distributed-xgboost-training-with-ray)
 * [How to Speed Up XGBoost Model Training](https://www.anyscale.com/blog/how-to-speed-up-xgboost-model-training)
 * [XGBoost-Ray Project](https://github.com/ray-project/xgboost_ray)
 * [Distributed XGBoost on Ray](Distributed XGBoost on Ray)